In [47]:
import os

cwd = os.getcwd()

In [48]:
import pandas as pd


df = pd.read_csv(cwd + '/data/posts_clean.csv')

In [49]:
df.head()

,Post Link,Title,Body,Tags,CreationDate,Answer Date,AcceptedAnswerId,id,score,complete_text,normalized_text
0,11227809,Why is processing a sorted array faster than p...,<p>Here is a piece of C++ code that shows some...,<java><c++><performance><cpu-architecture><bra...,2012-06-27 13:51:36,2012-06-27 13:56:42,11227902,11227902,26631,why is processing a sorted array faster than p...,process sort array faster process unsort array...
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,<git><version-control><git-commit><undo>,2009-05-29 18:09:14,2009-05-29 18:13:42,927386,927386,24839,how do i undo the most recent local commits in...,undo recent local commit git accident commit w...
2,2003505,How do I delete a Git branch locally and remot...,<h4>Failed Attempts to Delete a Remote Branch:...,<git><version-control><git-branch><git-push><g...,2010-01-05 01:12:15,2010-01-05 01:13:55,2003515,2003515,19584,how do i delete a git branch locally and remot...,delet git branch local remot fail attempt dele...
3,292357,What is the difference between 'git pull' and ...,"<p>What are the differences between <a href=""h...",<git><version-control><git-pull><git-fetch>,2008-11-15 09:51:09,2008-11-15 09:52:40,292359,292359,13375,what is the difference between git pull and gi...,differ git pull git fetch differ git pull git ...
4,231767,"What does the ""yield"" keyword do?",<p>What is the use of the <code>yield</code> k...,<python><iterator><generator>,2008-10-23 22:21:11,2008-10-23 22:48:44,231855,231855,12274,what does the yield keyword do what is the use...,yield keyword use yield keyword python exampl ...


## Criando os targets

In [50]:
def split_tags(tags):
    tags = tags[1:-1]
    tags = tags.split("><")
    
    return tags

In [51]:
df['targets'] = df.Tags.apply(lambda x: split_tags(x))

In [52]:
X, y = df['complete_text'], df['targets'].apply(lambda x: x[0])

In [53]:
y

0                  java
1                   git
2                   git
3                   git
4                python
              ...      
105598     unit-testing
105599                r
105600    asp.net-mvc-4
105601       postgresql
105602       javascript
Name: targets, Length: 105603, dtype: object

In [54]:
all_targets = df.targets.apply(lambda x: x[0]).tolist()

In [55]:
from itertools import chain


targets_set = list()

# for targets in all_targets:
#     for target in targets:
#         targets_set.append(target)
        
for targets in all_targets:
    targets_set.append(targets)

In [56]:
targets_set = set(targets_set)

In [57]:
len(targets_set)

1693

In [58]:
targets_set

{'raspberry-pi',
 'entity-framework-6',
 'gnu',
 'llvm',
 'jq',
 'wamp',
 'gitk',
 'computer-vision',
 'sublimetext',
 'zip',
 'mocking',
 'kernel',
 'postfix-mta',
 'windows-7-x64',
 'group-by',
 'jquery-mobile',
 'ecmascript-6',
 'lisp',
 'android',
 'monitoring',
 'google-image-search',
 'backup',
 'swagger-ui',
 'ip-address',
 'scalability',
 'kdiff3',
 'jekyll',
 'text-editor',
 'algorithm',
 'chef-infra',
 'varnish',
 'exec',
 'unix',
 'spam-prevention',
 'handler',
 'cordova',
 'programming-languages',
 'swift3',
 'tree',
 'plugins',
 'hidden-markov-models',
 'elixir',
 'mathematical-notation',
 'syntax-highlighting',
 'factory-pattern',
 'stdout',
 'webdriver',
 'mysql-workbench',
 'licensing',
 'gnu-make',
 'scale',
 'windbg',
 'c#-9.0',
 'utf-8',
 'azure-storage',
 'automation',
 'spinlock',
 'pymongo',
 'google-apps-script',
 'uiimage',
 'lvm',
 'ssas',
 'fortran',
 'jpa',
 'highcharts',
 'server-sent-events',
 'google-cloud-firestore',
 'ruby-on-rails',
 'perforce',
 'crt',

In [59]:
words_post = df.normalized_text.apply(lambda x: str(x))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer.fit(words_post)
vector = vectorizer.transform(words_post)

In [60]:
import gc

gc.collect()

0

In [61]:
from sklearn.model_selection import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(vector, y, test_size=0.2, random_state=42)

In [67]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(with_mean=False), LinearSVC(random_state=0, tol=1e-5, max_iter=10000))
clf.fit(X_train, y_train)


C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('linearsvc',
                 LinearSVC(max_iter=10000, random_state=0, tol=1e-05))])

In [68]:
y_pred = clf.predict(X_test)

array(['php', 'javascript', 'c#', ..., 'angular', 'html', 'angular'],
      dtype=object)

In [ ]:
gc.collect()

In [69]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[ 0  0  0 ...  0  0  0]
 [ 0 19  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                             precision    recall  f1-score   support

                  .htaccess       0.00      0.00      0.00         3
                       .net       0.14      0.11      0.13       166
                   .net-3.5       0.00      0.00      0.00         1
                   .net-4.0       0.00      0.00      0.00         1
                     .net-5       0.00      0.00      0.00         1
              .net-assembly       0.00      0.00      0.00         1
                  .net-core       0.00      0.00      0.00         1
             abstract-class       0.00      0.00      0.00         0
       abstract-syntax-tree       0.00      0.00      0.00         1
           access-modifiers       0.00      0.00      0.00         0
                        ack       0.00      0.00      0.00         0
             actionscript-3       0.00      0.00      0.00         1
           active-directory       0.00      0.00      0.00         1
                    ado.net      

C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Vitor\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

np.mean(cross_val_score(clf, X_train, y_train, cv=5))